<center> <h1> METHODE DE RECALAGE COHERENT POINT DRIFT </h1></center> 

# Importation des librairies 

In [1]:
from __future__ import print_function 
from __future__ import division  
import numpy as np
import math
from statistics import mean 
import matplotlib.pyplot as plt
import os
import cv2 as cv 
import sys
import cv2
from PIL import Image
from numpy import asarray
#from skimage.measure import structural_similarity as ssim
from skimage import measure
from skimage.metrics import structural_similarity as ssim
import time
import tarfile
import xml.dom.minidom as md
import skimage as ski # Scikit-Image : Image processing
import skimage.feature
import skimage.morphology
from skimage import img_as_float
from skimage.metrics import mean_squared_error as mse  #from skimage.measure import compare_mse as mse

import scipy.spatial.distance as ssd # Distance metric
from sklearn.cluster import KMeans
# For Coherent Point Drift (CPD) registration :
import pycpd
from pycpd import rigid_registration # CPD
from functools import partial
from socket import socket
# For mutual information registration :
from dipy.align.imaffine import AffineRegistration
from dipy.align.transforms import (RigidTransform2D, AffineTransform2D)
from dipy.viz import regtools
from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)

from pycpd import RigidRegistration
from PIL import Image as im 
#! pip install scikit-image
#! pip install dipy
#! pip install scipy
#! pip install pycpd
#! pip install skimage 
#! pip install cv2
#! pip install scikitlearn

C:\ProgramData\Anaconda3\lib\site-packages\dipy\viz\__init__.py:31: UserWarning: You do not have FURY installed. Some visualization functions might not work for you. For installation instructions, please visit: https://fury.gl/
  warnings.warn(


# Fonction de comparaison  MSE ET SSIM

In [2]:
def comparer_images(imageA, imageB):
    m = mse(imageA, imageB)
    s = ssim(imageA, imageB,multichannel=True,gaussian_weights=True)
    return m,s

# FONCTION DE RECALAGE CPD

In [17]:
def coherentpointdrift(img1_o,img2_o):
    """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    CPD : Coherent Point Drift method for Point registration and recalibration \n
    img1_o = Original first Image or Visible Image \n
    img2_o = Original second Image or Fluorescent Image \n
    Multimodal = logical, True, if the images are from different modalities (e.g. visible and fluorescent) \n
    Multistart = logical, True, if you want to test several initial conditions \n
    Sym = logical, True if you want to test symmetry with additional initial conditions \n
    Param = logical, True if you want to retrieve the transformation matrix and the position used for registration \n
    border = size of the border to add to avoid bad cropping \n
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
   # Load the data
    img1 = np.copy(img1_o)
    img2 = np.copy(img2_o)

    # Extract masks
    img1_m = ski.color.rgb2gray(img1) < 1 # img1 mask
    img2_m = ski.color.rgb2gray(img2) <1 # img2 mask

    # Extract contours with Canny filter
    img1_c = ski.feature.canny(img1_m) # Contour from first image
    img1_p = np.transpose(np.where(img1_c))  # Location of the contour points
    img2_c = ski.feature.canny(img2_m) # Contour from second image
    img2_p = np.transpose(np.where(img2_c))  # Location of the contour points
    
    # Callback function
    def trace(iteration, error,X,Y): # Function to see the progress of the registration
        print('Iteration: {:d}\nError: {:06.4f}'.format(iteration, error))
    callback = partial(trace) # Follow the progress of the rigid registration function
   
    # Background value
    background =[255,255,255]
    #reg =rigid_registration(**{ 'X': img1_p,'Y':img2_p,'max_iterations':200, 'tolerance':0.0001} )
    reg =RigidRegistration(**{ 'X': img1_p,'Y':img2_p,'max_iterations':50, 'tolerance':0.001} )
    reg.register(callback) # Rigid registration
    
    # Recalibration of every coordinates
    T = np.float32(np.c_[reg.R,[reg.t[1],reg.t[0]]]*reg.s) # Transformation matrix
    img2_new = cv2.warpAffine(img2,T,dsize=(img1.shape[1],img1.shape[0]),borderValue=background)

    # Output 
    return img2_new

# Application Coherent Point Drift 

In [25]:
n=35

temps=[]

for i in range(1,n+1):
    os.chdir (r'C:\Users\Adama\Desktop\Images_Plantes\feuille ({})'.format(i))
    image1=cv2.imread('Image (1).jpg') #image de reference
    image2=cv2.imread('Image (2).jpg') #image traitée
   
    #Appel des fonctions de recalage : ORB, MI
    t1 = time.time()
    imagerecalecp=coherentpointdrift(image1,image2)
    t2= time.time()

    # Calcul du TEMPS d'execution
    temps.append(t2-t1)
        
    #Enregistrement des images recalées convertie
    os.chdir (r'C:\Users\Adama\Desktop\Images_Plantes\feuille ({})'.format(i))
    cv.imwrite("imagerecalCP{}.jpg".format(i), imagerecalecp)
    

Iteration: 1
Error: 161774.0294
Iteration: 2
Error: 155242.7625
Iteration: 3
Error: 154560.2451
Iteration: 4
Error: 153606.0077
Iteration: 5
Error: 152255.9422
Iteration: 6
Error: 150365.6185
Iteration: 7
Error: 147773.1093
Iteration: 8
Error: 144342.3728
Iteration: 9
Error: 140020.7340
Iteration: 10
Error: 134878.9737
Iteration: 11
Error: 129133.0973
Iteration: 12
Error: 123070.5995
Iteration: 13
Error: 116971.1438
Iteration: 14
Error: 111108.6576
Iteration: 15
Error: 105722.1264
Iteration: 16
Error: 100969.8548
Iteration: 17
Error: 96970.9495
Iteration: 18
Error: 93836.7437
Iteration: 19
Error: 91596.4510
Iteration: 20
Error: 90131.4233
Iteration: 21
Error: 89233.1093
Iteration: 22
Error: 88702.1199
Iteration: 23
Error: 88393.5603
Iteration: 24
Error: 88215.3231
Iteration: 25
Error: 88112.3849
Iteration: 26
Error: 88052.7563
Iteration: 27
Error: 88018.0410
Iteration: 28
Error: 87997.6958
Iteration: 29
Error: 87985.6762
Iteration: 30
Error: 87978.5084
Iteration: 31
Error: 87974.1880
I

In [42]:
msecpd=[]
ssimcpd=[]
n=35

for i in range(1,n+1):
    os.chdir (r'C:\Users\Adama\Desktop\Images_Plantes\feuille ({})'.format(i))
    imagerecalecpd=cv2.imread("imagerecalCP{}.jpg".format(i),0) #image de reference
    image2=cv2.imread('Image (2).jpg',0) 
    msecpdi,simcpdi=comparer_images(imagerecalecpd, image2)
    msecpd.append(msecpdi)
    ssimcpd.append(simcpdi)
                   
#Affichage CPD
print('RMSE moyenne pour CPD : ' ,math.sqrt(mean(msecpd)))
print('SSIM moyenne pour CPD : ' ,mean(ssimcpd))
print('Temps moyenne dexecution pour CPD : ' ,mean(temps))

RMSE moyenne pour CPD :  26.41032285172112
SSIM moyenne pour CPD :  0.9765781029650686
Temps moyenne dexecution pour CPD :  487.879552595956
